In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import inv

In [ ]:
''' Basic TMM functions '''
### this function will compute the P matrix for a given layer l
### this function expects you have pre-calculated phil for that layer
def BuildP(phil):
    P = np.zeros((2,2),dtype=complex)
    ci = 0+1j

    a = -1*ci*phil
    b = ci*phil

    P[0][1] = 0+0j
    P[1][0] = 0+0j
    P[0][0] = np.exp(a)
    P[1][1] = np.exp(b)
    return P

def BuildD(nl, ctheta,pol):
    D = np.zeros((2,2),dtype=complex)


    if (pol=="s" or pol=="S"):
        D[0][0] = 1.+0j
        D[0][1] = 1.+0j
        D[1][0] = nl*ctheta
        D[1][1] = -1*nl*ctheta


    elif (pol=="p" or pol=="P"):
        D[0][0] = ctheta+0j
        D[0][1] = ctheta+0j
        D[1][0] = nl
        D[1][1] = -1*nl

    ### defaulting to P polarization 
    else:
        print("Polarization not chosen... defaulting to p-polarization")
        D[0][0] = ctheta+0j
        D[0][1] = ctheta+0j
        D[1][0] = nl
        D[1][1] = -1*nl

    return D

### main driver function for the transfer matrix method
### TMM should do the following:
### This assumes that nA and tA are both arrays of length equal
### to the number of layers
### 1. Calculate D_1^-1, which requires n[0] and theta0
### 2. Calculate D_2, D_2^-1, P_2 ... D_L-1, D_L-1^-1, P_L-1, which requires
###    phi_2 ... phi_L-1, k_z_2 ... k_z_L-1, d[1]...d[L-2]
### 3. Calculate D_L
### 4. Multiply all matrices together in order to form M
### 5. Return relevant quantity(s)... maybe M itself, maybe 1 - R - T... TBD

def tmm(k0, theta0, pol, nA, tA):
    t1 = np.zeros((2,2),dtype=complex)
    t2 = np.zeros((2,2),dtype=complex)
    D1 = np.zeros((2,2),dtype=complex)
    Dl = np.zeros((2,2),dtype=complex)
    Dli = np.zeros((2,2),dtype=complex)
    Pl = np.zeros((2,2),dtype=complex)
    M  = np.zeros((2,2),dtype=complex)
    L = len(nA)
    kz = np.zeros(L,dtype=complex)
    phil = np.zeros(L,dtype=complex)
    ctheta = np.zeros(L,dtype=complex)
    theta = np.zeros(L,dtype=complex)
    ctheta[0] = np.cos(theta0)

    D1 = BuildD(nA[0], ctheta[0], pol)
    ### Note it is actually faster to invert the 2x2 matrix
    ### "By Hand" than it is to use linalg.inv
    ### and this inv step seems to be the bottleneck for the TMM function
    tmp = D1[0,0]*D1[1,1]-D1[0,1]*D1[1,0]
    det = 1/tmp
    M[0,0] = det*D1[1,1]
    M[0,1] = -det*D1[0,1]
    M[1,0] = -det*D1[1,0]
    M[1,1] = det*D1[0,0]
    #D1i = inv(D1)
   #print("D1i is ")
   #print(D1i)


    ### This is the number of layers in the structure


    ### since kx is conserved through all layers, just compute it
    ### in the upper layer (layer 1), for which you already known
    ### the angle of incidence
    kx = nA[0]*k0*np.sin(theta0)
    kz[0] = np.sqrt((nA[0]*k0)**2 - kx**2)
    kz[L-1] = np.sqrt((nA[L-1]*k0)**2 - kx**2)
    ### keeping consistent with K-R excitation
    if np.real(kz[0])<0:
        kz[0] = -1*kz[0]
    if np.imag(kz[L-1])<0:
        kz[L-1] = -1*kz[L-1]
    ### loop through all layers 2 through L-1 and compute kz and cos(theta)...
    ### note that when i = 1, we are dealing with layer 2... when 
    ### i = L-2, we are dealing with layer L-1... this loop only goes through
    ### intermediate layers!
    for i in range(1,(L-1)):
        kz[i] = np.sqrt((nA[i]*k0)**2 - kx**2)
        if np.imag(kz[i])<0:
            kz[i] = -1*kz[i]

        ctheta[i] = kz[i]/(nA[i]*k0)
        theta[i] = np.arccos(ctheta[i])

        phil[i] = kz[i]*tA[i]

        Dl = BuildD(nA[i],ctheta[i], pol)
        ## Invert Dl
        tmp = Dl[0,0]*Dl[1,1]-Dl[0,1]*Dl[1,0]
        det = 1/tmp
        Dli[0,0] = det*Dl[1,1]
        Dli[0,1] = -det*Dl[0,1]
        Dli[1,0] = -det*Dl[1,0]
        Dli[1,1] = det*Dl[0,0]
        #Dli = inv(Dl)
        ## form Pl
        Pl = BuildP(phil[i])

        t1 = np.matmul(M,Dl)
        t2 = np.matmul(t1,Pl)
        M  = np.matmul(t2,Dli)

    ### M is now the product of D_1^-1 .... D_l-1^-1... just need to 
    ### compute D_L and multiply M*D_L
    kz[L-1] = np.sqrt((nA[L-1]*k0)**2 - kx**2)
    ctheta[L-1]= kz[L-1]/(nA[L-1]*k0)
    DL = BuildD(nA[L-1], ctheta[L-1], pol)
    t1 = np.matmul(M,DL)
    ### going to create a dictionary called M which will 
    ### contain the matrix elements of M as well as 
    ### other important quantities like incoming and outgoing angles
    theta[0] = theta0
    theta[L-1] = np.arccos(ctheta[L-1])
    ctheta[0] = np.cos(theta0)
    M = {"M11": t1[0,0],
         "M12": t1[0,1],
         "M21": t1[1,0],
         "M22": t1[1,1],
         "theta_i": theta0,
         "theta_L": np.real(np.arccos(ctheta[L-1])),
         "kz": kz,
         "phil": phil,
         "ctheta": ctheta,
         "theta": theta
         }

    return M




In [ ]:
''' TMM Gradient Functions '''
### this function will compute the derivative of the P matrix for a given
### layer l with respect to the thickness of layer l
def Build_dP_ds(kzl, dl):
    P = np.zeros((2,2),dtype=complex)
    ci = 0+1j
    a = -1*ci*kzl*dl
    b = ci*kzl*dl
    P[0][1] = 0+0j
    P[1][0] = 0+0j
    P[0][0] = -ci*kzl*np.exp(a)
    P[1][1] = ci*kzl*np.exp(b)
    return P

### analytically differentiates the transfer matrix
### with respect to the thickness of a layer li to be specified by 
### the user!
def tmm_grad(k0, theta0, pol, nA, tA, layers):
    n = len(layers)
    N = len(tA)
    ### Initialize arrays!
    Dli = np.zeros((N,2,2), dtype = complex)
    Dl = np.zeros((N,2,2), dtype = complex)
    D1 = np.zeros((2,2),dtype=complex)
    Pl = np.zeros((N, 2, 2), dtype = complex)
    Plp = np.zeros((n,2,2), dtype = complex)
    Mp = np.zeros((n, 2,2), dtype = complex)
    t1 = np.zeros((2,2), dtype = complex)
    t2 = np.zeros((2,2), dtype = complex)
    kz = np.zeros(N, dtype = complex)
    phil = np.zeros(N, dtype = complex)
    ctheta = np.zeros(N, dtype = complex)
    theta = np.zeros(N, dtype = complex)
    M = np.zeros((2,2), dtype = complex)
    ### compute kx
    kx = nA[0]*np.sin(theta0)*k0
    ### compute D1
    ctheta[0] = np.cos(theta0)
    D1 = BuildD(nA[0],ctheta[0], pol)
    ### compute D1^-1
    tmp = D1[0,0]*D1[1,1]-D1[0,1]*D1[1,0]
    det = 1/tmp
    M[0,0] = det*D1[1,1]
    M[0,1] = -det*D1[0,1]
    M[1,0] = -det*D1[1,0]
    M[1,1] = det*D1[0,0]
    Dli[0,:,:] = np.copy(M)
    ### Initialize gradient of M with D1^-1
    for i in range(0,n):
        Mp[i,:,:] = np.copy(M)
    ### Compute kz0
    kz[0] = np.sqrt(nA[0]*k0)**2-kx**2
    ### Compute kz, phil, D, P, D^-1 quantities for all  finite layers!
    for i in range (1,(N-1)):
        kz[i] = np.sqrt((nA[i]*k0)**2-kx**2)
        if np.imag(kz[i]) < 0:
            kz[i] = -1 * kz[i]
        ctheta[i] = kz[i]/(nA[i]*k0)
        theta[i] = np.arccos(ctheta[i])
        phil[i] = kz[i]*tA[i]
        Dl[i,:,:] = BuildD(nA[i], ctheta[i], pol)
        tmp = Dl[i,0,0]*Dl[i,1,1]-Dl[i,0,1]*Dl[i,1,0]
        det = 1/tmp
        Dli[i,0,0] = det*Dl[i,1,1]
        Dli[i,0,1] = -det*Dl[i,0,1]
        Dli[i,1,0] = -det*Dl[i,1,0]
        Dli[i,1,1] = det*Dl[i,0,0]
        Pl[i,:,:] = BuildP(phil[i])
        t1 = np.dot(M,Dl[i,:,:])
        t2 = np.dot(t1, Pl[i,:,:])
        M = np.dot(t2,Dli[i,:,:])

    ### kz, Dl for final layer!
    kz[N-1] = np.sqrt((nA[N-1]*k0)**2-kx**2)
    ctheta[N-1] = kz[N-1]/(nA[N-1]*k0)
    Dl[N-1,:,:] = BuildD(nA[N-1],ctheta[N-1], pol)
    t1 = np.dot(M,Dl[N-1,:,:])
    ### This is the transfer matrix!
    M = np.copy(t1)
    ### for all layers we want to differentiate with respect to, 
    ### form Plp matrices
    for l in range(0,n):
        Plp[l,:,:] = Build_dP_ds(kz[layers[l]],tA[layers[l]])
    ### for all those layers, compute associated matrix products!
    idx = 0
    for i in layers:
        for l in range(1,i):
            t1 = np.dot(Mp[idx,:,:], Dl[l,:,:])
            t2 = np.dot(t1, Pl[l,:,:])
            Mp[idx,:,:] = np.dot(t2,Dli[l,:,:])
        t1 = np.dot(Mp[idx,:,:],Dl[i,:,:])
        t2 = np.dot(t1,Plp[idx,:,:])
        Mp[idx,:,:] = np.dot(t2,Dli[i,:,:])
        for l in range(i+1,N-1):
            t1 = np.dot(Mp[idx,:,:], Dl[l,:,:])
            t2 = np.dot(t1, Pl[l,:,:])
            Mp[idx,:,:] = np.dot(t2,Dli[l,:,:])
        t1 = np.dot(Mp[idx,:,:],Dl[N-1,:,:])
        Mp[idx,:,:] = np.copy(t1)
        idx = idx+1

    M = {
         "Mp": Mp,
         "M11": M[0,0],
         "M12": M[0,1],
         "M21": M[1,0],
         "M22": M[1,1],
         "theta_i": theta0,
         "theta_L": np.real(np.arccos(ctheta[N-1])),
         "kz": kz,
         "phil": phil,
         "ctheta": ctheta,
         "theta": theta


            }
    return M

